# 01 - Exploratory Data Analysis - Initial EDA & Data Prep

(Python + tsforge)

**Goal** Get a fast, working understanding of our dataset *as a forecasting panel*, prove it’s modeling-ready, and produce a simple baseline forecast. We’ll keep it practical and direct:
- Load the prepared train/test splits (full + subset) from the setup.
- Run focused EDA for time series (frequency, completeness, zeros, outliers).
- Summarize/Pad/Visualize with `tsforge` wrappers (over Nixtla + pytimetk).


> We’re not polishing models here. We’re checking **data completeness**, **readiness**, and **signal health** 



In [36]:
from tsfeatures import tsfeatures

In [37]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)

ids = ["store_a", "store_b", "store_c"]
dept_map = {
    "store_a": "dept_1",
    "store_b": "dept_1",
    "store_c": "dept_2",
}

dates = pd.date_range(start="2020-01-01", periods=52 * 4, freq="W")

base_rates = {"store_a": 3.0, "store_b": 1.5, "store_c": 0.8}

records = []
for uid in ids:
    lam = base_rates[uid]
    counts = rng.poisson(lam=lam, size=len(dates))

    # introduce intermittent zero runs
    gap_mask = rng.choice([0, 1], size=len(dates), p=[0.2, 0.8])
    gap_lengths = rng.integers(1, 4, size=len(dates))
    idx = 0
    while idx < len(dates):
        if gap_mask[idx] == 0:
            counts[idx : idx + gap_lengths[idx]] = 0
            idx += gap_lengths[idx]
        else:
            idx += 1

    records.append(
        pd.DataFrame(
            {
                "unique_id": uid,
                "department": dept_map[uid],
                "ds": dates,
                "y": counts,
            }
        )
    )

id_df = pd.concat(records, ignore_index=True)


In [40]:
import pandas as pd 
import numpy as np
from tsforge.eda import hierarchical_tsfeatures,datetime_diagnostics,TSFORGE_FEATURES


date_table = datetime_diagnostics(
    df = id_df,
    id_col="unique_id",
    date_col="ds",
    target_col="y", # include target for peak seasons to be returned
)



In [39]:
dummy_df = hierarchical_tsfeatures(
    df = id_df,
    id_col='unique_id',
    date_col='ds',
    target_col='y',
    hierarchy=['unique_id','department'],
    features=TSFORGE_FEATURES,
    freq=52,
)

<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


In [41]:
dummy_df.head()

,permutation_entropy,MI_top_k_lags_indices,MI_top_k_lags,percentage_of_zeros,overdispersion,quarterly_MASE_score,yearly_MASE_score,monthly_MASE_score,hyndman_forecastability,number_of_trailing_zeros,...,crossing_points,arch_lm,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,hier_id
0,0.924562,"[25, 28, 29, 30, 31]",0.558724,35.096154,1.834675,1.266904,1.243764,1.172303,0.925780,0,...,84,0.042019,0.224588,0.127668,-0.338549,0.197491,-0.569496,0.370376,-0.024868,store_a
1,0.905015,"[25, 28, 29, 30, 31]",0.410650,46.634615,1.518858,1.002124,0.955583,1.037423,0.907034,2,...,89,0.057619,0.028881,0.017459,-0.504763,0.275144,-0.681607,0.526486,-0.001386,store_b
2,0.746518,"[51, 29, 30, 31, 32]",0.416338,65.865385,1.413491,1.109790,1.145979,1.001528,0.748053,1,...,88,0.024822,0.063542,0.038755,-0.485915,0.261282,-0.661113,0.504433,-0.063544,store_c
0,0.985704,"[51, 27, 28, 29, 30]",0.481154,17.788462,1.816039,1.086370,1.032051,1.087891,0.985989,0,...,92,0.029309,0.099902,0.072627,-0.433208,0.236663,-0.636463,0.468340,0.039849,dept_1
1,0.746518,"[51, 29, 30, 31, 32]",0.416338,65.865385,1.413491,1.109790,1.145979,1.001528,0.748053,1,...,88,0.024822,0.063542,0.038755,-0.485915,0.261282,-0.661113,0.504433,-0.063544,dept_2


In [42]:
date_table.head()

,start_date,end_date,n_obs,span_days,inferred_freq,obs_per_year,n_gaps,pct_missing,has_duplicates,peak_month,peak_quarter
unique_id,,,,,,,,,,,
store_a,2020-01-05,2023-12-24,208,1449.0,W-SUN,52.430642,0,0.0,False,3,1
store_b,2020-01-05,2023-12-24,208,1449.0,W-SUN,52.430642,0,0.0,False,2,1
store_c,2020-01-05,2023-12-24,208,1449.0,W-SUN,52.430642,0,0.0,False,8,3
